In [1]:
import predspracovanie

import os
import datetime

import numpy as np
import pandas as pd
import seaborn as sns
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

import pickle

import copy

import tensorflow.keras.backend as backend

from tensorflow.keras.regularizers import l2

import os

# os.environ["CUDA_DEVICE_ORDER"] = "GPU"  # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"] = "3"

from tensorflow.python.client import device_lib

print(device_lib.list_local_devices())

import os
import datetime

import wandb
from wandb.keras import WandbCallback


[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 7448976527568966066
, name: "/device:XLA_CPU:0"
device_type: "XLA_CPU"
memory_limit: 17179869184
locality {
}
incarnation: 9400944972637878405
physical_device_desc: "device: XLA_CPU device"
, name: "/device:XLA_GPU:0"
device_type: "XLA_GPU"
memory_limit: 17179869184
locality {
}
incarnation: 2184733191477737828
physical_device_desc: "device: XLA_GPU device"
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 7779456128
locality {
  bus_id: 2
  numa_node: 1
  links {
  }
}
incarnation: 11491398783758922116
physical_device_desc: "device: 0, name: GeForce GTX 1080, pci bus id: 0000:84:00.0, compute capability: 6.1"
]


In [2]:
items = [
    '30m-item56', 
         '30m-item57', '30m-item58', '30m-item59', '30m-item61', '30m-item62', '30m-item66', '30m-item67', '30m-item69', '30m-item71', '30m-item72', '30m-item73', '30m-item75',
    '30m-item76', 
    '30m-item77'
        ]
df = pickle.load( open( "/home/richard_stana/programing/clanok/data/picklnute/30min.p", "rb" ) )
# df = predspracovanie.make_dataset(df)
df = df.drop(["30m-item60", "30m-item63", "30m-item64", "30m-item65", "30m-item68", "30m-item70", "30m-item74"], axis=1)
# df = df[actual_item]
# predspracovanie.draw(df, datum_alebo_cisla="cislo", y1_orig=df[actual_item])

In [3]:
# train_df = df[76:44000]
# val_df = df[44001:46655]
# test_df = df[46800:52389]
train_df = df[27:15549]
train1_df = df[15602:17160]
val_df = df[17160-144-1:17460]
# test_df = df[46800:52389]

In [4]:
train_df_orig = train_df.copy()
train_df = predspracovanie.create_log_difference(train_df)
train1_df_orig = train1_df.copy()
train1_df = predspracovanie.create_log_difference(train1_df)
val_df_orig = val_df.copy()
val_df = predspracovanie.create_log_difference(val_df)
# test_df_orig = test_df.copy()
# test_df = predspracovanie.create_log_difference(test_df)

In [5]:
n_inputs = 144
n_outputs = 5

In [6]:
def compile_and_fit(projekt, model, nazov_modelu, train_x, train_y, train_x1, train_y1, val_x, val_y, compile=False, epochs=200):
    wandb.init(project=projekt, name=nazov_modelu + str(n_outputs))

    model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
        filepath="models_clanok/" + nazov_modelu + "future" + str(n_outputs) + ".hdf5",
        #     save_weights_only=True,
        monitor='val_mae',
        mode='min',
        save_best_only=True)
    model_checkpoint_callback1 = tf.keras.callbacks.ModelCheckpoint(
        filepath="models_clanok/" + nazov_modelu + "future" + str(n_outputs) + "1.hdf5",
        #     save_weights_only=True,
        monitor='val_mae',
        mode='min',
        save_best_only=True)

    if compile:
        model.compile(loss='mse',
                    optimizer=tf.optimizers.Adam(),
                    metrics='mae')

    history = model.fit(x=np.array(train_x).reshape((-1,n_inputs,1)),
                      y=np.array(train_y).reshape((-1,n_outputs)),
                      epochs=epochs,
                      validation_data=(np.array(val_x).reshape((-1,n_inputs,1)), np.array(val_y).reshape((-1,n_outputs))),
                      batch_size=128,
                      callbacks=[model_checkpoint_callback, 
                                     WandbCallback()
                                ])
    history = model.fit(x=np.array(train_x).reshape((-1,n_inputs,1)),
                      y=np.array(train_y).reshape((-1,n_outputs)),
                      epochs=epochs,
                      validation_data=(np.array(val_x).reshape((-1,n_inputs,1)), np.array(val_y).reshape((-1,n_outputs))),
                      batch_size=128,
                      callbacks=[model_checkpoint_callback1, 
                                     WandbCallback()
                                ])
    return history

In [7]:
lstm_input = keras.Input(
    shape=(n_inputs, 1), name="lstm_input"
)  
lstm = layers.LSTM(512, return_sequences=True)(lstm_input)
lstm = layers.LSTM(512, return_sequences=True)(lstm)
lstm = layers.LSTM(512, return_sequences=False)(lstm)
lstm_output = layers.Dense(units=n_outputs)(lstm)

In [8]:
model_lstm = keras.Model(
    inputs=[lstm_input],
    outputs=[lstm_output],
)

In [9]:
cnn_input = keras.Input(
    shape=(n_inputs, 1), name="cnn_input"
)  
cnn = layers.Conv1D(filters=256,
                           kernel_size=(7,),
                           activation='relu',
                           padding="same")(cnn_input)
cnn = layers.Conv1D(filters=256,
                           kernel_size=(7,),
                           activation='relu',
                           padding="same")(cnn)
cnn = layers.Conv1D(filters=256,
                           kernel_size=(7,),
                           activation='relu',
                           padding="same")(cnn)
flat = layers.Flatten()(cnn)
dense = layers.Dense(units=64, activation='relu')(flat)
cnn_output = layers.Dense(units=n_outputs)(dense)


In [10]:
model_cnn = keras.Model(
    inputs=[cnn_input],
    outputs=[cnn_output],
)

In [11]:
dobra_cnn_input = keras.Input(
    shape=(n_inputs, 1), name="dobra_cnn_input"
)  
dobra_cnn = layers.Conv1D(filters=256,
                           kernel_size=(7,),
                           activation='relu',
                           padding="same")(dobra_cnn_input)
dobra_cnn = layers.Conv1D(filters=128,
                           kernel_size=(5,),
                           activation='relu',
                           padding="same")(dobra_cnn)
dobra_cnn = layers.Conv1D(filters=64,
                           kernel_size=(3,),
                           activation='relu',
                           padding="same")(dobra_cnn)
dobra_flat = layers.Flatten()(dobra_cnn)
dobra_dense = layers.Dense(units=64, activation='relu')(dobra_flat)
dobra_cnn_output = layers.Dense(units=n_outputs)(dobra_dense)


In [12]:
dobra_model_cnn = keras.Model(
    inputs=[dobra_cnn_input],
    outputs=[dobra_cnn_output],
)

In [13]:
dobra_model_cnn.summary()

Model: "functional_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dobra_cnn_input (InputLayer) [(None, 144, 1)]          0         
_________________________________________________________________
conv1d_3 (Conv1D)            (None, 144, 256)          2048      
_________________________________________________________________
conv1d_4 (Conv1D)            (None, 144, 128)          163968    
_________________________________________________________________
conv1d_5 (Conv1D)            (None, 144, 64)           24640     
_________________________________________________________________
flatten_1 (Flatten)          (None, 9216)              0         
_________________________________________________________________
dense_3 (Dense)              (None, 64)                589888    
_________________________________________________________________
dense_4 (Dense)              (None, 5)                

In [14]:
down_up_cnn_input = keras.Input(
    shape=(n_inputs, 1), name="down_up_cnn_input"
)  
down_up_cnn = layers.Conv1D(filters=256,
                           kernel_size=(7,),
                           activation='relu',
                           padding="same")(down_up_cnn_input)
down_up_cnn = layers.Conv1D(filters=128,
                           kernel_size=(5,),
                           activation='relu',
                           padding="same")(down_up_cnn)
down_up_flat = layers.Flatten()(down_up_cnn)
down_up_dense = layers.Dense(units=1, activation='sigmoid')(down_up_flat)
down_up_dense = layers.Dense(units=144*64, input_dim=128)(down_up_dense)
down_up_reshape = layers.Reshape((144, 64))(down_up_dense)
down_up_cnn = layers.Conv1DTranspose(filters=128,
                           kernel_size=(5,),
                           activation='relu',
                           padding="same")(down_up_reshape)
down_up_cnn = layers.Conv1DTranspose(filters=256,
                           kernel_size=(7,),
                           activation='relu',
                           padding="same")(down_up_cnn)
down_up_flat = layers.Flatten()(down_up_cnn)
# down_up_dense = layers.Dense(units=64, activation='relu')(down_up_flat)
down_up_cnn_output = layers.Dense(units=n_outputs)(down_up_flat)

In [15]:
down_up_model_cnn = keras.Model(
    inputs=[down_up_cnn_input],
    outputs=[down_up_cnn_output],
)

In [16]:
down_up_model_cnn.summary()

Model: "functional_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
down_up_cnn_input (InputLaye [(None, 144, 1)]          0         
_________________________________________________________________
conv1d_6 (Conv1D)            (None, 144, 256)          2048      
_________________________________________________________________
conv1d_7 (Conv1D)            (None, 144, 128)          163968    
_________________________________________________________________
flatten_2 (Flatten)          (None, 18432)             0         
_________________________________________________________________
dense_5 (Dense)              (None, 1)                 18433     
_________________________________________________________________
dense_6 (Dense)              (None, 9216)              18432     
_________________________________________________________________
reshape (Reshape)            (None, 144, 64)          

In [17]:
by_lstm_input = keras.Input(
    shape=(n_inputs, 1), name="lstm_input"
)  
by_lstm = layers.Bidirectional(tf.keras.layers.LSTM(128, return_sequences=True, dropout=0.75, kernel_regularizer=l2(0.01), recurrent_regularizer=l2(0.01), bias_regularizer=l2(0.01)))(by_lstm_input)
by_lstm = layers.Bidirectional(tf.keras.layers.LSTM(128, return_sequences=True, dropout=0.75, kernel_regularizer=l2(0.01), recurrent_regularizer=l2(0.01), bias_regularizer=l2(0.01)))(by_lstm)
by_lstm = layers.Bidirectional(tf.keras.layers.LSTM(128, return_sequences=False, dropout=0.75, kernel_regularizer=l2(0.01), recurrent_regularizer=l2(0.01), bias_regularizer=l2(0.01)))(by_lstm)
by_lstm_output = layers.Dense(units=n_outputs)(by_lstm)

In [18]:
by_lstm_model = keras.Model(
    inputs=[by_lstm_input],
    outputs=[by_lstm_output],
)

In [19]:
gru_input = keras.Input(
    shape=(n_inputs, 1), name="gru_input"
)  
gru = layers.GRU(256, return_sequences=True, dropout=0.75, kernel_regularizer=l2(0.01), recurrent_regularizer=l2(0.01), bias_regularizer=l2(0.01))(gru_input)
gru = layers.GRU(256, return_sequences=True, dropout=0.75, kernel_regularizer=l2(0.01), recurrent_regularizer=l2(0.01), bias_regularizer=l2(0.01))(gru)
gru = layers.GRU(256, return_sequences=True, dropout=0.75, kernel_regularizer=l2(0.01), recurrent_regularizer=l2(0.01), bias_regularizer=l2(0.01))(gru)
simpleRRN = tf.keras.layers.SimpleRNN(128, dropout=0.75, kernel_regularizer=l2(0.01), recurrent_regularizer=l2(0.01), bias_regularizer=l2(0.01))(gru)
gru_output = layers.Dense(units=n_outputs)(simpleRRN)

In [20]:
gru_model = keras.Model(
    inputs=[gru_input],
    outputs=[gru_output],
)

In [21]:
encoder_inputs = keras.Input(
    shape=(n_inputs, 1), name="encoder_inputs"
)  
encoder_l1 = layers.LSTM(512, return_state=True)
encoder_outputs1 = encoder_l1(encoder_inputs)
encoder_states1 = encoder_outputs1[1:]
decoder_inputs = layers.RepeatVector(n_outputs)(encoder_outputs1[0])
decoder_l1 = layers.LSTM(512, return_sequences=True)(decoder_inputs,initial_state = encoder_states1)
decoder_outputs1 = layers.TimeDistributed(layers.Dense(1))(decoder_l1)

In [22]:
model_e1d1 = keras.Model(
    inputs=[encoder_inputs],
    outputs=[decoder_outputs1]
)
model_e1d1.summary()

Model: "functional_13"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
encoder_inputs (InputLayer)     [(None, 144, 1)]     0                                            
__________________________________________________________________________________________________
lstm_6 (LSTM)                   [(None, 512), (None, 1052672     encoder_inputs[0][0]             
__________________________________________________________________________________________________
repeat_vector (RepeatVector)    (None, 5, 512)       0           lstm_6[0][0]                     
__________________________________________________________________________________________________
lstm_7 (LSTM)                   (None, 5, 512)       2099200     repeat_vector[0][0]              
                                                                 lstm_6[0][1]         

In [23]:
dense_input = keras.Input(
    shape=(n_inputs), name="dense_input"
)  
dense = layers.Dense(units=1024, activation='relu')(dense_input)
dense = layers.Dense(units=512, activation='relu')(dense)
dense = layers.Dense(units=256, activation='relu')(dense)
dense = layers.Dense(units=128, activation='relu')(dense)
dense_output = layers.Dense(units=n_outputs)(dense)

In [24]:
model_dense = keras.Model(
    inputs=[dense_input],
    outputs=[dense_output]
)
model_dense.summary()

Model: "functional_15"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_input (InputLayer)     [(None, 144)]             0         
_________________________________________________________________
dense_11 (Dense)             (None, 1024)              148480    
_________________________________________________________________
dense_12 (Dense)             (None, 512)               524800    
_________________________________________________________________
dense_13 (Dense)             (None, 256)               131328    
_________________________________________________________________
dense_14 (Dense)             (None, 128)               32896     
_________________________________________________________________
dense_15 (Dense)             (None, 5)                 645       
Total params: 838,149
Trainable params: 838,149
Non-trainable params: 0
_______________________________________________

In [ ]:
for i in items:
    actual_item = i
    train_x, train_y = predspracovanie.create_x_y(train_df[actual_item], n_inputs, n_outputs)
    train_x1, train_y1 = predspracovanie.create_x_y(train1_df[actual_item], n_inputs, n_outputs)
    val_x, val_y = predspracovanie.create_x_y(val_df[actual_item], n_inputs, n_outputs)

    compile_and_fit(actual_item, model_lstm, "lstm-" + actual_item, train_x = train_x, train_y = train_y, train_x1 = train_x1, train_y1 = train_y1, val_x = val_x, val_y = val_y, compile=True, epochs=20)
    compile_and_fit(actual_item, model_cnn, "cnn-" + actual_item, train_x = train_x, train_y = train_y, train_x1 = train_x1, train_y1 = train_y1, val_x = val_x, val_y = val_y, compile=True, epochs=20)
    compile_and_fit(actual_item, dobra_model_cnn, "dobra_cnn-" + actual_item, train_x = train_x, train_y = train_y, train_x1 = train_x1, train_y1 = train_y1, val_x = val_x, val_y = val_y, compile=True, epochs=20)

    compile_and_fit(actual_item, down_up_model_cnn, "down_up_cnn-" + actual_item, train_x = train_x, train_y = train_y, train_x1 = train_x1, train_y1 = train_y1, val_x = val_x, val_y = val_y, compile=True, epochs=20)
    compile_and_fit(actual_item, by_lstm_model, "by_lstm-" + actual_item, train_x = train_x, train_y = train_y, train_x1 = train_x1, train_y1 = train_y1, val_x = val_x, val_y = val_y, compile=True, epochs=20)
    compile_and_fit(actual_item, gru_model, "gru-" + actual_item, train_x = train_x, train_y = train_y, train_x1 = train_x1, train_y1 = train_y1, val_x = val_x, val_y = val_y, compile=True, epochs=20)
    compile_and_fit(actual_item, model_e1d1, "e1d1-" + actual_item, train_x = train_x, train_y = train_y, train_x1 = train_x1, train_y1 = train_y1, val_x = val_x, val_y = val_y, compile=True, epochs=20)
    compile_and_fit(actual_item, model_dense, "dense-" + actual_item, train_x = train_x, train_y = train_y, train_x1 = train_x1, train_y1 = train_y1, val_x = val_x, val_y = val_y, compile=True, epochs=20)


Failed to query for notebook name, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable
wandb: Currently logged in as: kriza (use `wandb login --relogin` to force relogin)
wandb: wandb version 0.10.15 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


Epoch 1/20
121/121 [==============================] - 19s 154ms/step - loss: 0.0268 - mae: 0.0698 - val_loss: 0.0063 - val_mae: 0.0563
Epoch 2/20
121/121 [==============================] - 18s 148ms/step - loss: 0.0262 - mae: 0.0701 - val_loss: 0.0060 - val_mae: 0.0550
Epoch 3/20
121/121 [==============================] - 18s 146ms/step - loss: 0.0254 - mae: 0.0705 - val_loss: 0.0061 - val_mae: 0.0552
Epoch 4/20
121/121 [==============================] - 18s 147ms/step - loss: 0.0262 - mae: 0.0700 - val_loss: 0.0063 - val_mae: 0.0569
Epoch 5/20
121/121 [==============================] - 18s 147ms/step - loss: 0.0260 - mae: 0.0720 - val_loss: 0.0061 - val_mae: 0.0555
Epoch 6/20
121/121 [==============================] - 18s 148ms/step - loss: 0.0233 - mae: 0.0718 - val_loss: 0.0063 - val_mae: 0.0567
Epoch 7/20
121/121 [==============================] - 18s 148ms/step - loss: 0.0188 - mae: 0.0735 - val_loss: 0.0090 - val_mae: 0.0721
Epoch 8/20
121/121 [==============================] - 1